In [ ]:
# add validation tag to explanations based on scores

import json
from tqdm import tqdm

score_file = "../scores.json" 
data_file = "../llama_3.1_explanation_raw.jsonl"  
output_file = "../lama-8b_explanation_with_validation.jsonl"

with open(score_file, "r") as f:
    score_data = json.load(f)
score_data = {k: float(v) for k, v in score_data.items()}

with open(data_file, "r") as f:
    all_data = [json.loads(line) for line in f]

for instance in tqdm(all_data):
    inst_id = instance["id"]
    new_comments = []
    for idx, (reason_text, label_code) in enumerate(instance["generated_explanations"]):
        reason_id = f"{inst_id}_{label_code}-{idx}"
        if reason_id in score_data and score_data[reason_id] > 0.5:
            tag = "validated"
        elif reason_id in score_data and score_data[reason_id] <= 0.5:
            tag = "not_validated"
        else:
            print("missing:", reason_id)

        new_comments.append([reason_text, label_code, tag])
    instance["generated_explanations"] = new_comments

with open(output_file, "w") as f:
    for instance in all_data:
        f.write(json.dumps(instance, ensure_ascii=False) + "\n")

print("Done.")


In [ ]:
# add label-level counts

import json
from collections import defaultdict
from tqdm import tqdm

input_file = "../lama-8b_explanation_with_validation.jsonl"
output_file = "../lama-8b_explanation_with_validation_count.jsonl"

with open(input_file, "r") as f:
    all_data = [json.loads(line) for line in f]

for instance in tqdm(all_data):
    comments = instance.get("generated_explanations", [])

    count_r1 = defaultdict(int)
    count_r2 = defaultdict(int)

    for reason_text, label_code, tag in comments:
        count_r1[label_code] += 1  
        if tag == "validated":
            count_r2[label_code] += 1 

    set_r1 = set(count_r1.keys())
    set_r2 = set(count_r2.keys())

    instance["label_count_round_1"] = dict(count_r1)
    instance["label_set_round_1"] = list(set_r1)

    instance["label_count_round_2"] = dict(count_r2)
    instance["label_set_round_2"] = list(set_r2)

 
    instance["error"] = list(set_r1 - set_r2)
    not_validated_exp = {
        label: count_r1[label] - count_r2.get(label, 0)
        for label in count_r1
        if count_r1[label] - count_r2.get(label, 0) > 0
    }
    instance["not_validated_exp"] = not_validated_exp

with open(output_file, "w") as f:
    for instance in all_data:
        f.write(json.dumps(instance, ensure_ascii=False) + "\n")

print("Done.")

In [ ]:
# error comparison between LLM validation and VariErr dataset

import json
import csv
from tqdm import tqdm

file_a_path = "../lama-8b_explanation_with_validation_count.jsonl"
file_b_path = "../varierr.json"
output_csv_path = "../merged_errors.csv"

with open(file_a_path, "r") as f:
    data_a = {json.loads(line)["id"]: json.loads(line) for line in f}

with open(file_b_path, "r") as f:
    data_b = {json.loads(line)["id"]: json.loads(line) for line in f}

all_ids = list(data_a.keys())

with open(output_csv_path, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["id", "llm_not_calidated_error", "varierr_error"])
    writer.writeheader()

    for id_ in tqdm(all_ids):
        if id_ not in data_b:
            print(f"ID {id_} not found in VariErr dataset.")
        
        row = {
            "id": id_,
            "llm_not_calidated_error": json.dumps(data_a.get(id_, {}).get("error", []), ensure_ascii=False),
            "varierr_error": json.dumps(data_b.get(id_, {}).get("error_labels", []), ensure_ascii=False)
        }
        writer.writerow(row)

print(f"CSV saved to：{output_csv_path}")


In [ ]:
# JSD between model and human distributions_chaosnli

import json
import numpy as np
import pandas as pd
from scipy.spatial.distance import jensenshannon
from pathlib import Path

# label_map = {"e": "entailment", "n": "neutral", "c": "contradiction"}
label_order = ["e", "n", "c"]


def normalize(counts):
    arr = np.array(counts, dtype=np.float64)
    if arr.sum() == 0:
        return np.ones_like(arr) / len(arr)
    return arr / arr.sum()


def load_model_counts(file_path):
    dist_map = {}
    with open(file_path, "r") as f:
        for line in f:
            data = json.loads(line)
            uid = data["id"]
            label_counts = data.get("label_count_round_1", {})  # change here 1 or 2

            dist_map[uid] = {
                "e": float(label_counts.get("e", 0) or 0),
                "n": float(label_counts.get("n", 0) or 0),
                "c": float(label_counts.get("c", 0) or 0)
            }

    return dist_map

def load_human_counts(file_path):
    with open(file_path, "r") as f:
        records = [json.loads(line) for line in f]

    dist_map = {}
    for r in records:
        uid = r["uid"]
        counter = r["label_counter"]
        dist_map[uid] = {k: v for k, v in counter.items()}
    
    # print(dist_map)
    return dist_map

def kl_divergence(p, q, eps=1e-12):
    p = np.array(p, dtype=np.float64) + eps
    q = np.array(q, dtype=np.float64) + eps
    return np.sum(p * np.log2(p / q))


def compare_distributions(model_counts, human_counts):
    rows = []
    shared_ids = set(model_counts.keys()) & set(human_counts.keys())
    # print(shared_ids)

    for uid in shared_ids:
        model_vec = [model_counts[uid].get(label, 0) for label in label_order]
        human_vec = [human_counts[uid].get(label, 0) for label in label_order]
        # print(f"uid: {uid}, model_vec: {model_vec}, human_vec: {human_vec}")

        model_dist = normalize(model_vec)
        human_dist = normalize(human_vec)
      
        jsd = jensenshannon(model_dist, human_dist, base=2.0) ** 2
        kl = kl_divergence(model_dist, human_dist)

        rows.append({
            "uid": uid,
            "js_divergence": jsd,
            "kl_divergence": kl,
            "model_distribution": model_dist.tolist(),
            "human_distribution": human_dist.tolist(),
            "model_counts": model_vec,
            "human_counts": human_vec,
        })

    return rows


def main(model_file, human_file, output_csv):
    model_counts = load_model_counts(model_file)
    human_counts = load_human_counts(human_file)

    comparison = compare_distributions(model_counts, human_counts)
    df = pd.DataFrame(comparison)
    df.to_csv(output_csv, index=False)
    print(f"Saved.")


if __name__ == "__main__":
    model_file = "../lama-8b_explanation_with_validation_count.jsonl"    
    human_file = "../chaosNLI_mnli_m.jsonl"    
    output_file = "../chaos_llama_8b_jsd_kl_before.csv"
    main(model_file, human_file, output_file)


In [ ]:
# JSD between model and human distributions_varierr

import json
import numpy as np
import pandas as pd
from scipy.spatial.distance import jensenshannon
from pathlib import Path

# label_map = {"e": "entailment", "n": "neutral", "c": "contradiction"}
label_order = ["e", "n", "c"]


def normalize(counts):
    arr = np.array(counts, dtype=np.float64)
    if arr.sum() == 0:
        return np.ones_like(arr) / len(arr)
    return arr / arr.sum()


def load_model_counts(file_path):
    dist_map = {}
    with open(file_path, "r") as f:
        for line in f:
            data = json.loads(line)
            uid = data["id"]
            label_counts = data.get("label_count_round_1", {})  # change here 1 or 2

            dist_map[uid] = {
                "e": float(label_counts.get("e", 0) or 0),
                "n": float(label_counts.get("n", 0) or 0),
                "c": float(label_counts.get("c", 0) or 0)
            }

    return dist_map


def load_human_counts(file_path):
    with open(file_path, "r") as f:
        records = [json.loads(line) for line in f]  

    dist_map = {}
    for r in records:
        uid = r["id"]
        raw_counts = r.get("label_count_round_1", {})   # change here 1 or 2

        counter = {
            "e": float(raw_counts.get("entailment") or 0),
            "n": float(raw_counts.get("neutral") or 0),
            "c": float(raw_counts.get("contradiction") or 0)
        }

        dist_map[uid] = counter

    print(dist_map)
    return dist_map

def kl_divergence(p, q, eps=1e-12):
    p = np.array(p, dtype=np.float64) + eps
    q = np.array(q, dtype=np.float64) + eps
    return np.sum(p * np.log2(p / q))


def compare_distributions(model_counts, human_counts):
    rows = []
    shared_ids = set(model_counts.keys()) & set(human_counts.keys())
    print(f"number of shared IDs:",len(shared_ids))

    for uid in shared_ids:
        model_vec = [model_counts[uid].get(label, 0) for label in label_order]
        human_vec = [human_counts[uid].get(label, 0) for label in label_order]
        # print(f"uid: {uid}, model_vec: {model_vec}, human_vec: {human_vec}")

        model_dist = normalize(model_vec)
        human_dist = normalize(human_vec)

        jsd = jensenshannon(model_dist, human_dist, base=2.0) ** 2
        kl = kl_divergence(model_dist, human_dist)

        rows.append({
            "uid": uid,
            "js_divergence": jsd,
            "kl_divergence": kl,
            "model_distribution": model_dist.tolist(),
            "human_distribution": human_dist.tolist(),
            "model_counts": model_vec,
            "human_counts": human_vec,
        })

    return rows


def main(model_file, human_file, output_csv):
    model_counts = load_model_counts(model_file)
    human_counts = load_human_counts(human_file)

    comparison = compare_distributions(model_counts, human_counts)
    df = pd.DataFrame(comparison)
    df.to_csv(output_csv, index=False)
    print("Saved.")


if __name__ == "__main__":
    model_file = "../lama-8b_explanation_with_validation_count.jsonl" 
    human_file = "../varierr.json"   
    output_file = "../varierr_llama_8b_jsd_kl_before.csv"
    main(model_file, human_file, output_file)


In [ ]:
# calculate average score
import pandas as pd

csv_file = "../dist_eval/chaos_llama_8b_jsd_kl_before.csv"

df = pd.read_csv(csv_file)
mean_jsd = df["js_divergence"].mean()
mean_kld = df["kl_divergence"].mean()

print(f"Average JSD：{mean_jsd:.4f}")
print(f"Average KLD：{mean_kld:.4f}")
